In [ ]:
import os
from pathlib import Path
import torch
from transformers import AutoTokenizer, PretrainedConfig
from diffusers import (
    AutoencoderKL,
    ControlNetModel,
    DDPMScheduler,
    StableDiffusionControlNetPipeline,
    UNet2DConditionModel,
    UniPCMultistepScheduler,
)

from neuraltexture_controlnet import NeuralTextureControlNetModel

In [ ]:
controlnet_dir = "runs_512/dilightnet-openillum-2-1-2-base-v2-8bit/checkpoint-27000/controlnet"
val_jsonl_path = (
    "dataset_512_v1/eval_v2.jsonl"
)
# num_res_blocks = 4

In [ ]:
def import_model_class_from_model_name_or_path(
    pretrained_model_name_or_path: str, revision: str
):
    text_encoder_config = PretrainedConfig.from_pretrained(
        pretrained_model_name_or_path,
        subfolder="text_encoder",
        revision=revision,
    )
    model_class = text_encoder_config.architectures[0]

    if model_class == "CLIPTextModel":
        from transformers import CLIPTextModel

        return CLIPTextModel

    else:
        raise ValueError(f"{model_class} is not supported.")



def init_pipeline(controlnet_dir):
    text_encoder_cls = import_model_class_from_model_name_or_path(
        "stabilityai/stable-diffusion-2-1-base", None
    )
    unet = UNet2DConditionModel.from_pretrained(
        "stabilityai/stable-diffusion-2-1-base",
        subfolder="unet",
    )
    weight_dtype = torch.float32
    # Load scheduler and models
    noise_scheduler = DDPMScheduler.from_pretrained(
        "stabilityai/stable-diffusion-2-1-base", subfolder="scheduler"
    )
    text_encoder = text_encoder_cls.from_pretrained(
        "stabilityai/stable-diffusion-2-1-base",
        subfolder="text_encoder",
    )
    vae = AutoencoderKL.from_pretrained(
        "stabilityai/stable-diffusion-2-1-base", subfolder="vae"
    )
    tokenizer = AutoTokenizer.from_pretrained(
        "stabilityai/stable-diffusion-2-1-base",
        subfolder="tokenizer",
        use_fast=False,
    )

    controlnet = NeuralTextureControlNetModel.from_pretrained(
        controlnet_dir, torch_dtype=weight_dtype
    )

    pipeline = StableDiffusionControlNetPipeline.from_pretrained(
        "stabilityai/stable-diffusion-2-1-base",
        tokenizer=tokenizer,
        unet=unet,
        controlnet=controlnet,
        safety_checker=None,
        torch_dtype=weight_dtype,
    )
    pipeline.scheduler = UniPCMultistepScheduler.from_config(pipeline.scheduler.config)
    pipeline = pipeline.to("cuda")
    # pipeline.eval()
    unet.requires_grad_(False)
    text_encoder.requires_grad_(False)
    vae.requires_grad_(False)
    controlnet.requires_grad_(False)
    return pipeline

In [ ]:
def generate_validation_images(pipe,val_json_path, export_dir, num_inference_steps = 100, cfg_weight = 7.5, num_batches=5):
    from relighting_dataset import RelightingDataset

    validation_dataset = RelightingDataset(
        data_jsonl=val_json_path,
        pretrained_model= "stabilityai/stable-diffusion-2-1-base",
        channel_aug_ratio= 0,  # add to args
        empty_prompt_ratio= 0,  # add to args
        log_encode_hint=False,  # add to args
        load_mask=True,  # add to args
    )

    validation_ref_images = []
    output_file_names = []
    validation_prompts = []

    for i in range(len(validation_dataset)):
        sample = validation_dataset[i]
        validation_prompts.append(sample["text"])
        validation_ref_images.append(sample["conditioning_pixel_values"].to('cuda'))
        source_file_name = sample["ref_file"] # "dataset_512_v1/eval/obj_60-fabric-green-hedgehog/CF8/013/gt.png"
        target_file_name = sample["target_file"] # e.g. "dataset_512_v1/eval/obj_60-fabric-green-hedgehog/CF8/010/gt.png"
        object_id = source_file_name.split("/")[2]
        source_light_id = source_file_name.split("/")[-2]
        target_light_id = target_file_name.split("/")[-2]
        view_id = source_file_name.split("/")[-3]
        output_file_name = f"{object_id}_src_{source_light_id}_tgt_{target_light_id}_{view_id}.png"
        output_file_names.append(output_file_name)

    # Generate images
    # pipe.eval()
    for k in range(num_batches):
        dir_to_save = os.path.join(export_dir, f"batch_{k}")
        os.makedirs(dir_to_save, exist_ok=True)

        generated_images = pipe(
            validation_prompts,
            validation_ref_images,
            num_inference_steps= num_inference_steps,
            cfg_weight=cfg_weight
        ).images

        for i, image in enumerate(generated_images):
            image.save(os.path.join(dir_to_save, output_file_names[i]))
            print(f"Saved image {os.path.join(dir_to_save, output_file_names[i])}")


In [ ]:
pipeline = init_pipeline(controlnet_dir)
generate_validation_images(
    pipeline,
    val_jsonl_path,
    f"generated_images_{controlnet_dir.split('/')[-1]}",
    num_inference_steps=100,
    cfg_weight=7.5,
    num_batches=5,
)

In [ ]:
import os
import yaml
import argparse
import json
import glob 
from pathlib import Path
from datetime import datetime
from typing import Literal, Union

from PIL import Image 
import numpy as np
import torch

import lpips
from cleanfid import fid
from torchmetrics.image import PeakSignalNoiseRatio
from torchmetrics.image import StructuralSimilarityIndexMeasure


def get_current_time():
    now = datetime.now().strftime("%m-%d-%H%M%S")
    return now


def pil_to_torch(img):
    img = np.array(img).astype(np.float32) / 255.0
    img = torch.from_numpy(img).permute(2, 0, 1).unsqueeze(0)
    return img

class Evaluator:
    def __init__(
        self,
        fdir1: Union[str, os.PathLike] = None,
        fdir2: Union[str, os.PathLike] = None,
        save_dir: Union[str, os.PathLike] = None,
        device: int = 0,
    ):
        self.fdir1 = fdir1
        self.fdir2 = fdir2
        self.save_dir = save_dir

        # Load evaluation metrics
        self.device = device
        self.PSNR = PeakSignalNoiseRatio().to(self.device)
        self.SSIM = StructuralSimilarityIndexMeasure().to(self.device)
        self.VGG = lpips.LPIPS(net='vgg').to(self.device)


    def __call__(self):
        fdir1 = self.fdir1
        fdir2 = self.fdir2
        save_dir = self.save_dir

        assert Path(fdir1).exists(), f"{fdir1} not exist."
        assert Path(fdir2).exists(), f"{fdir2} not exist."

        # Compute FID: set-to-set
        fid_score = fid.compute_fid(fdir1, fdir2)
        
        # Compute PSNR, SSIM, LPIPNS. image-to-image.
        # Make sure the corresponding images have the same filename in both directories. 
        # assert len(glob.glob(f"{fdir1}/*")) == len(glob.glob(f"{fdir2}/*")), f"{len(glob.glob(f'{fdir1}/*'))} != {len(glob.glob(f'{fdir2}/*'))}"

        psnr_score = []
        ssim_score = []
        lpips_score = []
        for img_path in glob.glob(f"{fdir1}/*"):
            fn = img_path.split("/")[-1]
            src_img_path = img_path
            tgt_img_path = os.path.join(fdir2, fn)
            assert os.path.exists(src_img_path), src_img_path
            assert os.path.exists(tgt_img_path), tgt_img_path

            img1 = Image.open(src_img_path).convert("RGB")
            img2 = Image.open(tgt_img_path).convert("RGB")

            torch_img1 = pil_to_torch(img1).to(self.device) # (1, 3, H, W), [0, 1]
            torch_img2 = pil_to_torch(img2).to(self.device)
            assert torch_img2.shape == torch_img1.shape, f"{torch_img1.shape}, {torch_img2.shape}"
            assert torch_img1.max() <= 1.0 and torch_img1.min() >= 0.0

            psnr_score.append(self.PSNR(torch_img1, torch_img2).item())
            ssim_score.append(self.SSIM(torch_img1, torch_img2).item())
            lpips_score.append(self.VGG(torch_img1, torch_img2).item())

        N = len(psnr_score)
        metric_dict = {
            "fdir1": fdir1,
            "fdir2": fdir2,
            "FID": fid_score,
            "PSNR": sum(psnr_score) / N,
            "SSIM": sum(ssim_score) / N,
            "LPIPS": sum(lpips_score) / N,
        }

        Path(save_dir).mkdir(exist_ok=True)
        now = get_current_time()
        metric_path = Path(save_dir) / f"eval_result_{now}.json"
        with open(metric_path, "w") as f:
            json.dump(metric_dict, f, indent=4)

        print(metric_dict)
        print(f"[*] Logged at {metric_path}")
        return metric_dict

    def cherrypick(self, root_dir, dist_func = "LPIPS", export_dir = None):
        """
        Cherry pick the best images from the generated images.

        Args:
            root_dir (str): The root directory of the generated images. 
            The directory structure should follow this structure:
            ```
            root_dir
            ├── batch_0
            │   ├── image1.png
            │   ├── image2.png
            │   └── ...
            ├── batch_1
            │   ├── image1.png
            │   ├── image2.png
            │   └── ...
            └── ...
            ```
        """
        assert dist_func in ["PSNR", "SSIM", "LPIPS"], f"{dist_func} not supported."
        if dist_func == "PSNR":
            dist_func = lambda x, y: -self.PSNR(x, y).item()
        elif dist_func == "SSIM":
            dist_func = lambda x, y: -self.SSIM(x, y).item()
        else:
            dist_func = lambda x, y: self.VGG(x, y).item()

        batch_dirs = os.listdir(root_dir)
        batch_dirs = [os.path.join(root_dir, d) for d in batch_dirs]
        
        # self.fdir1  is the ground truth directory. there are multiple images in the directory.
         # files in the fdir1
        gt_images = os.listdir(self.fdir1)
        
        best_img_files = []
        best_dists = []
        for gt_image in gt_images:
            gt_image_path = os.path.join(self.fdir1, gt_image)
            gt_img = Image.open(gt_image_path).convert("RGB")
            min_dist = float("inf")
            best_img_file = None
            for batch_dir in batch_dirs:
                batch_img = os.path.join(batch_dir, gt_image)
                assert os.path.exists(batch_img), f"{batch_img} not exist."
                gen_img = Image.open(batch_img).convert("RGB")
                torch_gt_img = pil_to_torch(gt_img).to(self.device)
                torch_gen_img = pil_to_torch(gen_img).to(self.device)
                assert torch_gt_img.shape == torch_gen_img.shape, f"{torch_gt_img.shape}, {torch_gen_img.shape}"
                assert torch_gt_img.max() <= 1.0 and torch_gt_img.min() >= 0.0
                dist_score = dist_func(torch_gt_img, torch_gen_img)
                
                if dist_score < min_dist:
                    min_dist = dist_score
                    best_img_file = batch_img
            best_img_files.append(best_img_file)
            best_dists.append(min_dist)
        
        if export_dir is not None:
            Path(export_dir).mkdir(exist_ok=True)
            for i, img_file in enumerate(best_img_files):
                img = Image.open(img_file)
                img.save(os.path.join(export_dir, f"{os.path.basename(img_file)}"))
                print(f"Saved {os.path.join(export_dir, f'{os.path.basename(img_file)}')}")
        return best_img_files, best_dists
    


import pandas as pd

# Metric dict의 리스트를 DataFrame으로 변환하는 함수
def metrics_to_dataframe(metric_dicts):
    # 필요한 키만 추출해서 DataFrame 생성
    data = [
        {
            "cfg_scale": metric_dict['fdir2'].split('_')[-1],  # fdir2에서 cfg_scale 추출
            "FID": metric_dict['FID'],
            "PSNR": metric_dict['PSNR'],
            "SSIM": metric_dict['SSIM'],
            "LPIPS": metric_dict['LPIPS']
        }
        for metric_dict in metric_dicts
    ]
    df = pd.DataFrame(data)
    # cfg_scale을 숫자로 변환
    df['cfg_scale'] = df['cfg_scale'].astype(float)
    # 소수점 네자리로 반올림
    return df.round({'FID': 4, 'PSNR': 4, 'SSIM': 4, 'LPIPS': 4})


In [ ]:
evaluator = Evaluator(
    fdir1="eval_gt",
    fdir2="generated_512_1/batch_1",
    save_dir="eval_result",
    device=0,
)


In [13]:
# cfgs = [0.1, 0.3, 0.5, 0.7]
cfgs = [0.1, 0.3, 0.5, 0.7, 0.9, 1.0, 1.2, 1.4]
dicts = []
for cfg in cfgs:
    imgs, scores = evaluator.cherrypick(f'generated_images/generated_128_4ksteps_cfg_{cfg}', export_dir = f'4ksteps-psnr_best_images_128_{cfg}', dist_func="PSNR")
    evaluator_2 = Evaluator(
        fdir1="eval_gt",
        fdir2=f'4ksteps-psnr_best_images_128_{cfg}',
        save_dir="eval_result-4ksteps",
        device=0,
    )
    res_dict = evaluator_2()
    dicts.append(res_dict)

df = metrics_to_dataframe(dicts)
df.to_csv("eval_result-4ksteps.csv", index=False)
df

Saved 4ksteps-psnr_best_images_128_0.1/obj_26_pumpkin_src_008_tgt_004_NE7.png
Saved 4ksteps-psnr_best_images_128_0.1/obj_11_duck_src_001_tgt_009_CB5.png
Saved 4ksteps-psnr_best_images_128_0.1/obj_62-fabric-birthday-cake_src_003_tgt_011_NE1.png
Saved 4ksteps-psnr_best_images_128_0.1/obj_08_transparent_cup_src_013_tgt_011_CE2.png
Saved 4ksteps-psnr_best_images_128_0.1/obj_35_transparent_plastic_cup_src_002_tgt_013_CF8.png
Saved 4ksteps-psnr_best_images_128_0.1/obj_20_red_pot_src_009_tgt_011_CA2.png
Saved 4ksteps-psnr_best_images_128_0.1/obj_64_greenhead_src_011_tgt_009_NE1.png
Saved 4ksteps-psnr_best_images_128_0.1/obj_03_chicken_src_002_tgt_013_CB5.png
Saved 4ksteps-psnr_best_images_128_0.1/obj_29_fabric_toy_src_009_tgt_004_CF8.png
Saved 4ksteps-psnr_best_images_128_0.1/obj_51_plastic_man_src_008_tgt_005_CB5.png
Saved 4ksteps-psnr_best_images_128_0.1/obj_42_food_banana_src_005_tgt_008_CE2.png
Saved 4ksteps-psnr_best_images_128_0.1/obj_31_painted_toy_src_007_tgt_001_CF8.png
Saved 4ksteps

/home/work/miniconda3/envs/train/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/work/miniconda3/envs/train/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
/home/work/miniconda3/envs/train/lib/python3.10/site-packages/lpips/lpips.py:107: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See http

Loading model from: /home/work/miniconda3/envs/train/lib/python3.10/site-packages/lpips/weights/v0.1/vgg.pth
compute FID between two folders
Found 64 images in the folder eval_gt


FID eval_gt : 100%|██████████| 2/2 [00:01<00:00,  1.51it/s]


Found 64 images in the folder 4ksteps-psnr_best_images_128_0.1


...

FID eval_gt : 100%|██████████| 2/2 [00:01<00:00,  1.61it/s]


Found 64 images in the folder 4ksteps-psnr_best_images_128_1.4


FID 4ksteps-psnr_best_images_128_1.4 : 100%|██████████| 2/2 [00:00<00:00,  2.12it/s]


{'fdir1': 'eval_gt', 'fdir2': '4ksteps-psnr_best_images_128_1.4', 'FID': np.float64(82.62351424001554), 'PSNR': 27.03463214635849, 'SSIM': 0.8883028924465179, 'LPIPS': 0.05715623805008363}
[*] Logged at eval_result-4ksteps/eval_result_11-30-181144.json


,cfg_scale,FID,PSNR,SSIM,LPIPS
0,0.1,77.5733,27.0373,0.8910,0.0557
1,0.3,76.8502,27.0049,0.8892,0.0555
2,0.5,78.7889,27.1187,0.8932,0.0559
3,0.7,80.4115,27.0623,0.8924,0.0552
4,0.9,75.8638,27.2378,0.8912,0.0552
5,1.0,76.6195,27.2990,0.8905,0.0546
6,1.2,81.4726,27.2984,0.8942,0.0545
7,1.4,82.6235,27.0346,0.8883,0.0572
